# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

boston = datasets.load_boston()

def populate_table(tablename, feature_names):
    X = boston.data  
    y = boston.target
    N = X.shape[0]
    y = y.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : list(boston.feature_names),
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/boston.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/boston.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TGT
230,230,230,0.53700,0.0,6.20,0,0.504,5.981,68.1,3.6715,8,307,17.4,378.35,11.65,24
112,112,112,0.12329,0.0,10.01,0,0.547,5.913,92.9,2.3534,6,432,17.8,394.95,16.21,18
125,125,125,0.16902,0.0,25.65,0,0.581,5.986,88.4,1.9929,2,188,19.1,385.02,14.81,21
9,9,9,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10,18
213,213,213,0.14052,0.0,10.59,0,0.489,6.375,32.3,3.9454,4,277,18.6,385.81,9.38,28
109,109,109,0.26363,0.0,8.56,0,0.520,6.229,91.2,2.5451,5,384,20.9,391.23,15.55,19
127,127,127,0.25915,0.0,21.89,0,0.624,5.693,96.0,1.7883,4,437,21.2,392.11,17.19,16
244,244,244,0.20608,22.0,5.86,0,0.431,5.593,76.5,7.9549,7,330,19.1,372.49,12.50,17
406,406,406,20.71620,0.0,18.10,0,0.659,4.138,100.0,1.1781,24,666,20.2,370.22,23.34,11
490,490,490,0.20746,0.0,27.74,0,0.609,5.093,98.0,1.8226,4,711,20.1,318.43,29.68,8


## Training a Model

In [5]:


# train any scikit model on the iris dataset

import lightgbm as lgb


clf = lgb.LGBMRegressor(objective='regression',
                         num_leaves=40,
                         learning_rate=0.05,
                         n_estimators=512)


clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


/home/antoine/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LGBMRegressor(learning_rate=0.05, n_estimators=512, num_leaves=40,
              objective='regression')

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	namespace LGBM_Tree_0 {
	
		typedef std::vector<double> tNodeData;
		std::map<int, tNodeData> Decision_Tree_Node_data = {
				{ 7 ,  {23.247226632527212 }} ,
				{ 8 ,  {22.48197020029541 }} ,
				{ 12 ,  {22.726893263748984 }} ,
				{ 13 ,  {22.41826827047267 }} ,
				{ 20 ,  {21.988059946557335 }} ,
				{ 22 ,  {21.697529627211953 }} ,
				{ 39 ,  {21.964448836429288 }} ,
				{ 43 ,  {21.688059930693807 }} ,
				{ 46 ,  {21.603536119325092 }} ,
				{ 47 ,  {21.40889326357559 }} ,
				{ 72 ,  {22.201393277227083 }} ,
				{ 73 ,  {22.391501977571505 }} ,
				{ 74 ,  {22.029726614217413 }} ,
				{ 76 ,  {22.104347826301936 }} ,
				{ 84 ,  {21.78162054388891 }} ,
				{ 85 ,  {21.89828721885281 }} ,
				{ 150 ,  {22.135083756981423 }} ,
				{ 151 ,  {22.206393280896496 }} ,
				{ 154 ,  {22.063241106969684 }} ,
				{ 155 ,  {21.991788017110203 }} 
		};
		
	
		int get_decision_tree_node_index(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3, st

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:100])
        print(result2[-100:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/boston2.csv" , ["Feature_" + str(i) for i,x in enumerate(metadata["features"])])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/boston2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_140459210833056.exe /tmp/sklearn2sql_cpp_140459210833056.cpp'
idx,Estimator
0,23.95077369657054
1,21.06971054119132
2,34.00136427309075
3,32.93024741716793
4,36.0
64095035984
502,19.52084473461657
503,23.08488940068428
504,21.67038709790006
505,11.90035724884689



In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Estimator
230,230,23.957918
112,112,17.998575
125,125,20.873060
9,9,17.936772
213,213,27.889822
109,109,18.887930
127,127,15.854848
244,244,17.005282
406,406,13.235720
490,490,8.463113


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_score = pd.DataFrame(clf.predict(X), columns=['Estimator']);
skl_output = pd.concat([skl_output_key, skl_output_score] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Estimator
230,230,23.957918
112,112,17.998575
125,125,20.873060
9,9,17.936772
213,213,27.889822
109,109,18.887930
127,127,15.854848
244,244,17.005282
406,406,13.235720
490,490,8.463113


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Estimator_skl,idx_cpp,Estimator_cpp
230,230,23.957918,230,23.957918
112,112,17.998575,112,17.998575
125,125,20.873060,125,20.873060
9,9,17.936772,9,17.936772
213,213,27.889822,213,27.889822
109,109,18.887930,109,18.887930
127,127,15.854848,127,15.854848
244,244,17.005282,244,17.005282
406,406,13.235720,406,13.235720
490,490,8.463113,490,8.463113


In [14]:
lDiff = cpp_skl_join['Estimator_skl'] - cpp_skl_join['Estimator_cpp']


In [15]:
lDiff.describe()

count    5.060000e+02
mean    -7.021173e-17
std      3.324866e-15
min     -7.105427e-15
25%     -3.552714e-15
50%      0.000000e+00
75%      3.552714e-15
max      7.105427e-15
dtype: float64